In [1]:
#!pip install qiskit
#!pip install qiskit-aer
#already installed in the environment

In [2]:
# Deutsch Algorithm using Qiskit 2.x
# Compatible with Qiskit 2.0+ (2024–2025)

from qiskit import QuantumCircuit, transpile
from qiskit.quantum_info import Statevector
from qiskit_aer import AerSimulator

# Choose the function type: 'constant_0', 'constant_1', 'balanced_0', 'balanced_1'
function_type = 'balanced_1'

def deutsch_oracle(qc, function_type):
    """Implements oracle Uf for given function type."""
    if function_type == 'constant_0':
        # f(x)=0 → Do nothing
        pass
    elif function_type == 'constant_1':
        # f(x)=1 → Apply X on the output qubit
        qc.x(1)
    elif function_type == 'balanced_0':
        # f(x)=x → Apply CNOT (control: input, target: output)
        qc.cx(0, 1)
    elif function_type == 'balanced_1':
        # f(x)=NOT(x) → Apply X, then CNOT, then X
        qc.x(0)
        qc.cx(0, 1)
        qc.x(0)

In [3]:
# Step 1: Initialize quantum circuit with 2 qubits and 1 classical bit
qc = QuantumCircuit(2, 1)

# Step 2: Initialize |x>|y> = |0>|1> and apply Hadamard
qc.x(1)             # Set output qubit to |1>
qc.barrier()
qc.h([0, 1])        # Apply Hadamard to both qubits

# Step 3: Apply the oracle
qc.barrier()
deutsch_oracle(qc, function_type)

In [4]:
# Step 4: Apply Hadamard to input qubit
qc.barrier()
qc.h(0)

# Step 5: Measure the first qubit
qc.measure(0, 0)

# Visualize circuit
print(qc.draw(output="text"))

           ░ ┌───┐ ░ ┌───┐     ┌───┐ ░ ┌───┐┌─┐
q_0: ──────░─┤ H ├─░─┤ X ├──■──┤ X ├─░─┤ H ├┤M├
     ┌───┐ ░ ├───┤ ░ └───┘┌─┴─┐└───┘ ░ └───┘└╥┘
q_1: ┤ X ├─░─┤ H ├─░──────┤ X ├──────░───────╫─
     └───┘ ░ └───┘ ░      └───┘      ░       ║ 
c: 1/════════════════════════════════════════╩═
                                             0 


In [5]:
# Step 6: Simulate
sim = AerSimulator()
qc_compiled = transpile(qc, sim)
result = sim.run(qc_compiled).result()
counts = result.get_counts()

print("\nMeasurement results:", counts)

# Interpret result
if list(counts.keys())[0] == '0':
    print("→ Function is CONSTANT.")
else:
    print("→ Function is BALANCED.")


Measurement results: {'1': 1024}
→ Function is BALANCED.


Task 1

In [6]:
# --- Custom Oracle: f(x) = x XOR 1 ---
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

def custom_oracle_xor1(qc):
    """Implements oracle for f(x) = x XOR 1"""
    qc.x(0)        # Flip input (x -> x ⊕ 1)
    qc.cx(0, 1)    # Apply CNOT
    qc.x(0)        # Restore input

# Initialize circuit
qc = QuantumCircuit(2, 1)
qc.x(1)
qc.h([0, 1])
qc.barrier()

custom_oracle_xor1(qc)
qc.barrier()
qc.h(0)
qc.measure(0, 0)

# Simulate
sim = AerSimulator()
qc_compiled = transpile(qc, sim)
result = sim.run(qc_compiled).result()
counts = result.get_counts()
print("Measurement results:", counts)

if list(counts.keys())[0] == '0':
    print("→ Function is CONSTANT.")
else:
    print("→ Function is BALANCED.")


Measurement results: {'1': 1024}
→ Function is BALANCED.


Task 2

In [7]:
# --- Bloch Sphere Visualization (Output-Only Version) ---
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, partial_trace

# Step 1: Create circuit and apply Hadamard
qc = QuantumCircuit(2)
qc.x(1)
qc.h([0, 1])
state_after_h = Statevector(qc)

print("Statevector after Hadamard:\n", state_after_h.data)

# Step 2: Apply Oracle (f(x) = NOT(x))
qc.barrier()
qc.x(0)
qc.cx(0, 1)
qc.x(0)
state_after_oracle = Statevector(qc)

print("\nStatevector after Oracle:\n", state_after_oracle.data)

# Step 3: Each qubit's reduced state (density matrices)
rho_q0 = partial_trace(state_after_oracle, [1])  # trace out qubit 1
rho_q1 = partial_trace(state_after_oracle, [0])  # trace out qubit 0

print("\nReduced density matrix for Qubit 0:\n", rho_q0.data)
print("\nReduced density matrix for Qubit 1:\n", rho_q1.data)

Statevector after Hadamard:
 [ 0.5+0.j  0.5+0.j -0.5+0.j -0.5+0.j]

Statevector after Oracle:
 [-0.5+0.j  0.5+0.j  0.5+0.j -0.5+0.j]

Reduced density matrix for Qubit 0:
 [[ 0.5+0.j -0.5+0.j]
 [-0.5+0.j  0.5+0.j]]

Reduced density matrix for Qubit 1:
 [[ 0.5+0.j -0.5+0.j]
 [-0.5+0.j  0.5+0.j]]


Task 3

In [9]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator

def oracle_dj_2qubit(qc, function_type="balanced"):
    """Example 2-qubit oracle: balanced or constant"""
    if function_type == "constant":
        pass
    elif function_type == "balanced":
        # Flip output for half inputs (e.g. when x1 XOR x2 = 1)
        qc.cx(0, 2)
        qc.cx(1, 2)

# Build 3-qubit circuit (2 input + 1 output)
qc = QuantumCircuit(3, 2)
qc.x(2)
qc.h([0, 1, 2])
qc.barrier()

oracle_dj_2qubit(qc, "balanced")
qc.barrier()
qc.h([0, 1])
qc.measure([0, 1], [0, 1])

# Simulate
sim = AerSimulator()
qc_compiled = transpile(qc, sim)
result = sim.run(qc_compiled).result()
counts = result.get_counts()
print("Measurement results:", counts)

if '00' in counts:
    print("→ Function is CONSTANT.")
else:
    print("→ Function is BALANCED.")

Measurement results: {'11': 1024}
→ Function is BALANCED.


Task 4

In [10]:
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit import transpile

# Create a simple depolarizing noise model
noise_model = NoiseModel()
# 1-qubit error for single-qubit gates like 'h'
error_1q = depolarizing_error(0.05, 1)
# 2-qubit error for two-qubit gates like 'cx'
error_2q = depolarizing_error(0.05, 2)

noise_model.add_all_qubit_quantum_error(error_1q, ['h'])
noise_model.add_all_qubit_quantum_error(error_2q, ['cx'])


# Reuse your Deutsch circuit
from qiskit import QuantumCircuit
qc = QuantumCircuit(2, 1)
qc.x(1)
qc.h([0, 1])
qc.cx(0, 1)
qc.h(0)
qc.measure(0, 0)

sim_noise = AerSimulator(noise_model=noise_model)
qc_compiled = transpile(qc, sim_noise)
result = sim_noise.run(qc_compiled, shots=1000).result()
counts = result.get_counts()
print("Noisy results:", counts)

Noisy results: {'1': 928, '0': 72}


Task 5

In [11]:
# Classical evaluation of f(x)
def f(x):
    return (x + 1) % 2  # Example balanced function

def classical_check():
    # Evaluate both inputs: 0 and 1
    f0, f1 = f(0), f(1)
    print(f"f(0)={f0}, f(1)={f1}")
    if f0 == f1:
        print("→ Classical: CONSTANT")
    else:
        print("→ Classical: BALANCED")
    print("Classical queries used:", 2)

# Quantum algorithm uses only 1 query
print("\nClassical method:")
classical_check()
print("\nQuantum method:")
print("→ Quantum: BALANCED (only 1 query used)")


Classical method:
f(0)=1, f(1)=0
→ Classical: BALANCED
Classical queries used: 2

Quantum method:
→ Quantum: BALANCED (only 1 query used)
